In [2]:

#HJI
import urllib3
import json


In [6]:
f = open("조커리뷰.txt",'r',encoding='UTF-8')
형태소카운터 = {}
name = f.name

In [7]:
switch = True
while(switch):
    댓글뭉치 = ''
    for i in range(50):
        text = f.readline()
        댓글뭉치 += text
        
    if text == '':
        switch = False
    elif text == '\n':
        continue
    else:
        openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
        accessKey = "3314db0e-ee18-4aff-bd16-e85e2a503569"
        analysisCode = "srl"
        text = 댓글뭉치

        requestJson = {
            "access_key": accessKey,
            "argument": {
                "text": text,
                "analysis_code": analysisCode
            }
        }

        http = urllib3.PoolManager()
        response = http.request(
            "POST",
            openApiURL,
            headers={"Content-Type": "application/json; charset=UTF-8"},
            body=json.dumps(requestJson)
        )

        a = response.data.decode('utf-8')
        result = json.loads(a)
        
        for l in result['return_object']['sentence']:
            for i in l['morp']: 
                if i['lemma'] not in 형태소카운터.keys() and (i['type']=='NNP' or i['type']=='NNG') :
                    형태소카운터[i['lemma']] = [1,i['type']]
                elif i['lemma'] in 형태소카운터.keys() and (i['type']=='NNP' or i['type']=='NNG'):
                    형태소카운터[i['lemma']][0]+=1

